In [8]:
from sys import path
path.insert(0, '../')

from news_api import api_requests,const

In [86]:
import pandas as pd
from tqdm import tqdm
from time import sleep
from urllib.parse import urlparse
from datetime import datetime

pd.set_option('display.max_colwidth',150)

In [17]:
allnews = {}

for country_code in tqdm(const.COUNTRIES):
    try:
        country_news = api_requests.get_articles(country_code)
    except Exception as err:
        country_news = []
        print(f'{country_code} failed:')
        
    allnews[country_code] = country_news
    sleep(1)

100%|█████████████████████████████████████████████████████████████████████████████████| 54/54 [01:05<00:00,  1.21s/it]


In [66]:
allnews_flatten = []

for ctr,nws in allnews.items():
    for n in nws:
        n['country'] = ctr
        allnews_flatten.append(n)

allnews_flatten = pd.DataFrame(allnews_flatten)

In [67]:
def website_parser(url):
    return urlparse(url).netloc
    
allnews_flatten['source_website'] = allnews_flatten['url'].apply(website_parser)
allnews_flatten = allnews_flatten[['country', 'source_website', 'author', 'title', 'publishedAt']]

In [71]:
allnews_flatten['source_website'].value_counts()[:10]

source_website
news.google.com                931
www.youtube.com                  9
timesofindia.indiatimes.com      5
www.nu.nl                        5
www.chosun.com                   4
www.ndtv.com                     4
www.bbc.com                      4
www.chinanews.com.cn             3
www.hani.co.kr                   3
www3.nhk.or.jp                   3
Name: count, dtype: int64

In [72]:
allnews_flatten['author'].value_counts()[:10]

author
Aktuality.sk      8
DELFI             7
CNA               7
The Guardian      7
Novinky.cz        7
CNN en Español    7
Yahoo奇摩新聞         7
infobae           7
RTV Slovenija     6
Le Monde          6
Name: count, dtype: int64

In [88]:
date_version = datetime.today().strftime('%d%m%y-%Hh%M')

allnews_flatten.to_csv(f'data/news_export_{date_version}.csv')